In [ ]:
#@title First check what GPU you got and make sure it's a good one. 
#@markdown - Tier List: (K80 < T4 < P100 < V100 < A100)
from subprocess import getoutput
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gputil ffpb
import GPUtil as GPU
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
gpu = GPU.getGPUs()[0]
!nvidia-smi -L

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
GPU 0: Tesla T4 (UUID: GPU-4d1b611f-8f2b-e53d-b051-0e0ea28ab919)


In [ ]:

!nvidia-smi

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Mon Jul 18 08:54:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## installation
!pip install ftfy==5.8 transformers
!pip install git+https://github.com/openai/CLIP.git --no-deps
!pip install sentence_transformers
!pip install kornia
!pip install git+https://github.com/eps696/aphantasia
!pip install omegaconf>=2.0.0 torchmetrics==0.6.2 pytorch-lightning>=1.0.8 einops>=0.3.0
!git clone https://github.com/CompVis/taming-transformers
!mv taming-transformers/* ./
!pip install streamlit
!pip install moviepy
!pip install ffmpeg
!pip install gtts
!pip install langdetect
!pip install Pillow
!pip install arabic-reshaper
!pip install python-bidi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xh_215td
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-xh_215td
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/eps696/aphantasia to /tmp/pip-req-build-jzu0x0lv
  Running command git clone -q https://github.com/eps696/aphantasia /tmp/pip-req-build-jzu0x0lv
fatal: destination path 'taming-transformers' already exists and is not an empty directory.
mv: cannot stat 'taming-transformers/*': No such file or directory
Lo

In [ ]:
print("Downloading CLIP and installing requirements")
!git clone https://github.com/openai/CLIP                 &> /dev/null
!pip install -q  ftfy regex tqdm omegaconf pytorch-lightning fastprogress kornia &> /dev/null

In [ ]:
!pip install -q siren-pytorch # install the library

In [ ]:
#@title imports and utility functions for CLIP and such
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

from tqdm.notebook import tqdm

import kornia.augmentation as K
from CLIP import clip
from torchvision import transforms as tf

from PIL import Image
import numpy as np
import math

from matplotlib import pyplot as plt
from fastprogress.fastprogress import master_bar, progress_bar
from IPython.display import HTML
from base64 import b64encode

import warnings
warnings.filterwarnings('ignore') # Some pytorch functions give warnings about behaviour changes that I don't want to see over and over again :)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]

class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1 

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch

def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)

class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply 

In [ ]:
import re
import os
import cv2
import matplotlib.pyplot as plt
import glob
import PIL
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import *
from gtts import gTTS
import imageio
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import kornia
from aphantasia.utils import slice_imgs, txt_clean, old_torch
from aphantasia import transforms
import pytorch_lightning as pl
import yaml
from omegaconf import OmegaConf
import clip
from sentence_transformers import SentenceTransformer
from taming.modules.diffusionmodules.model import Decoder
from taming.modules.vqvae.quantize import VectorQuantizer2 as VectorQuantizer
from taming.modules.vqvae.quantize import GumbelQuantize
from langdetect import detect
import arabic_reshaper
from bidi.algorithm import get_display

In [ ]:
import torch
from torch import nn
from siren_pytorch import Sine, Siren, SirenNet, SirenWrapper

In [ ]:
class VQModel(pl.LightningModule):
  def __init__(self, ddconfig, n_embed, embed_dim, remap=None, sane_index_shape=False, **kwargs_ignore):  # tell vector quantizer to return indices as bhw
    super().__init__()
    self.decoder = Decoder(**ddconfig)
    self.quantize = VectorQuantizer(n_embed, embed_dim, beta=0.25, remap=remap, sane_index_shape=sane_index_shape)
  def decode(self, quant):
    return self.decoder(quant)

class GumbelVQ(VQModel):
  def __init__(self, ddconfig, n_embed, embed_dim, kl_weight=1e-8, remap=None, **kwargs_ignore):
    z_channels = ddconfig["z_channels"]
    super().__init__(ddconfig, n_embed, embed_dim)
    self.quantize = GumbelQuantize(z_channels, embed_dim, n_embed=n_embed, kl_weight=kl_weight, temp_init=1.0, remap=remap)

class latents(torch.nn.Module):
  def __init__(self, shape):
    super(latents, self).__init__()
    init_rnd = torch.zeros(shape).normal_(0.,4.)
    self.lats = torch.nn.Parameter(init_rnd.cuda())
  def forward(self):
    return self.lats

def load_config(VQGAN_model, config_path):
  config = OmegaConf.load(config_path)
  return config

def load_vqgan(VQGAN_model, config, ckpt_path=None):
  if VQGAN_model == "gumbel_f8-8192":
    model = GumbelVQ(**config.model.params)
  else:
    model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model.eval()

def vqgan_image(model, z):
  x = model.decode(z)
  x = (x+1.)/2.
  return x

def enc_text(txt, model_clip, model_lang, multilang):
  if multilang:
    emb = model_lang.encode([txt], convert_to_tensor=True, show_progress_bar=False)
  else:
    emb = model_clip.encode_text(clip.tokenize(txt).cuda())
  return emb.detach().clone()

img_array = []

def save_img(img, txt, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1, 2, 0))  
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  img_array.append(img)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave(txt + '.jpg', np.array(img))

def checkout(num, txt, model_vqgan, lats, tempdir):
  with torch.no_grad():
    img = vqgan_image(model_vqgan, lats()).cpu().numpy()[0]
  save_img(img, txt, os.path.join(f"/content/{tempdir}", '%04d.jpg' % num))

def train(i, txt, model_vqgan, lats, tempdir, save_freq, samples, modsize, trform_f, model_clip, txt_enc, sign, optimizer, align, style, txt_enc2 , prompt , wrapper , normalize , make_cutouts):
  loss = 0
  img_out = vqgan_image(model_vqgan, lats())
  img_sliced = slice_imgs([img_out], samples, modsize, trform_f, align, macro=0.4)[0]
  out_enc = model_clip.encode_image(img_sliced)
  if len(text) > 0:
    loss += sign * torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
  if len(style) > 0:
    loss += sign * 0.5 * torch.cosine_similarity(txt_enc2, out_enc, dim=-1).mean()
  del img_out, img_sliced, out_enc; torch.cuda.empty_cache()
  # Track out losses
  losses = []
  # The optimization loop
  for i in tqdm(range(50)):
    # Zero grads
    optimizer.zero_grad()
    # get image
    im = wrapper().to(device)
    # encode with CLIP  
    iii = model_clip.encode_image(normalize(make_cutouts(im))).float()
    # Calculate loss  
    loss = prompt(iii)
    # Store
    losses.append(loss.item())
    # Step optim
    loss.backward()
    optimizer.step()
  # Show loss plot and final result
  fig, axs = plt.subplots(1, 2, figsize=(16, 6))
  axs[0].plot(losses) 
  im = wrapper()
  im_arr = np.array(255 * (im.squeeze().permute(1, 2, 0).detach())).astype(np.uint8)
  axs[1].imshow(im_arr)
  if i % save_freq == 0:
    checkout(i // save_freq, txt, model_vqgan, lats, tempdir)
  return lats, im_arr

def create_images(text, sideY, sideX, scale_res, model_clip, model_lang, multilang, learning_rate, steps, model_vqgan, save_freq, samples, modsize, trform_f, sign, txt_enc2, align , normalize , make_cutouts):
  images = []
  workdir = 'images_samples'
  os.makedirs(os.path.join("./" , "final_images"), exist_ok=True)
  for i in range(len(text)):
    tempdir = os.path.join(workdir, '%s%s' % (txt_clean(text[i])[:100], txt_clean(style)[:50]))
    os.makedirs(tempdir, exist_ok=True)
    lats = latents([1, 256, sideY//scale_res, sideX//scale_res]).cuda()
    txt_enc = enc_text(text[i], model_clip, model_lang, multilang)
    prompt = Prompt(txt_enc, 1, float('-inf')).to(device)
    # A network with multiple layers:
    net = SirenNet(
        dim_in = 2,                        # input dimension, ex. 2d coordinates
        dim_hidden = 64,                  # hidden dimension
        dim_out = 3,                       # output dimension, ex. rgb value
        num_layers = 5,                    # number of layers
        final_activation = nn.Sigmoid(),   # activation of final layer (nn.Identity() for direct output)
        w0_initial = 30.                   # different signals may require different omega_0 in the first layer - this is a hyperparameter
    )

    # The wrapper for rendering an image
    wrapper = SirenWrapper(
        net,
        image_width = 256,
        image_height = 256
    )
    # The optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3) # I just guessed a learning rate
    # optimizer = torch.optim.AdamW(lats.parameters(), learning_rate, weight_decay=0.01, amsgrad=True)
    temp = lats
    for j in range(steps):
      new_lats, image = train(j, text[i], model_vqgan, lats, tempdir, save_freq, samples, modsize, trform_f, model_clip, txt_enc, sign, optimizer, align, style, txt_enc2 , prompt , wrapper , normalize , make_cutouts)
      temp = new_lats
    image = PIL.Image.fromarray(image)
    image_path = "/content/final_images/" + f"{text[i]}.jpg"
    image.save(image_path)
    images.append(image_path)
  return images

def write_on_images(images, text, lang):
  os.makedirs(os.path.join("./" , "images_with_texts"), exist_ok=True)
  for i in range(len(images)):
    image = open(images[i], 'rb')
    img = PIL.Image.open(image)
    width, height = img.size
    draw = ImageDraw.Draw(img)
    if lang == 'ar':
      sentence = arabic_reshaper.reshape(text[i])
      sentence = get_display(sentence)
    else:
      sentence = text[i]
    font = ImageFont.truetype("/content/drive/MyDrive/GP/fonts/Amiri-Bold.ttf", 25)
    w, h = font.getsize(sentence)
    Coords = ((width-w)/2, height-h-5)
    color = (255, 0, 0)
    draw.text(Coords, sentence, color, font=font)
    img.save("/content/images_with_texts/" + f'pic_text{i}.jpg')

def create_audios(text, lang):
  os.makedirs("audio_files", exist_ok=True)
  for i in range(len(text)):
    myobj = gTTS(text=text[i], lang=lang, slow=False)
    myobj.save(f"/content/audio_files/audio{i}.mp3")
  audio_paths = glob.glob(f"/content/audio_files/*.mp3")
  audio_paths = sorted(audio_paths)
  return audio_paths

def create_videos(audio_paths):
  os.makedirs("video_files", exist_ok=True)
  videos_path = "/content/video_files"
  images_texts = glob.glob(f"/content/images_with_texts/*.jpg")
  images_texts = sorted(images_texts)
  for i in range(len(images_texts)):
    add_static_image_to_audio(images_texts[i], audio_paths[i], f"{videos_path}/video{i}.mp4")
  video_clip_paths = glob.glob(f"/content/video_files/*.mp4")
  video_clip_paths = sorted(video_clip_paths)
  return video_clip_paths

def add_static_image_to_audio(image_path, audio_path, output_path):
    audio_clip = AudioFileClip(audio_path)
    image_clip = ImageClip(image_path)
    video_clip = image_clip.set_audio(audio_clip)
    video_clip.duration = audio_clip.duration
    video_clip.fps = 1
    video_clip.write_videofile(output_path, fps=60)

def concatenate_videos(video_clip_paths):
  os.makedirs("final_video", exist_ok=True)
  video_path = "/content/final_video/finalVideo.mp4"
  clips = [VideoFileClip(c) for c in video_clip_paths]
  final_clip = concatenate_videoclips(clips)
  final_clip.write_videofile(video_path, codec="libx264", fps=60)
  return video_path

def run_model(text, style):
  # paraameters
  lang = detect(text)
  sideX = 300
  sideY = 300
  steps = 100
  samples = 60
  learning_rate = 0.1
  save_freq = 1
  scale_res = 8
  sign = -1.
  align = "uniform" # 'uniform', 'central', 'overscan'
  aug_transform = "custom"  # 'custom', 'elastic', 'fast'
  
  multilang = False
  if lang != "en":
    multilang = True
  
  text = re.sub(r"[;:.!?]", ',', text)
  text  = text.split(',')
  for txt in text:
    if txt == '':
      text.remove(txt)

  if not os.path.isdir('/content/models_TT'):
    !mkdir -p /content/models_TT
  def getm(url, path):
    if os.path.isfile(path) and os.stat(path).st_size > 0: 
      print(' already exists', path, os.stat(path).st_size)
    else:
      !wget $url -O $path
  # models
  VQGAN_model = "imagenet_f16-16384" 
  # VQGAN_model = "gumbel_f8-8192" 
  # available clip vision transformers: ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
  model = "ViT-B/32" # sideX:700, sideY:500, steps:200, samples:60, scale_res:8
  # model = "ViT-L/14"   # sideX:700, sideY:300, steps:200, samples:65, scale_res:8 
  # model = "ViT-L/14@336px"  # sideX:300, sideY:300, steps:200, samples:40, scale_res:8
  # if VQGAN_model == "imagenet_f16-16384" and not os.path.isfile('/content/models_TT/imagenet_f16-16384.ckpt'):
  #   getm('https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1', '/content/models_TT/imagenet_f16-16384.ckpt')
  #   getm('https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1', '/content/models_TT/imagenet_f16-16384.yaml')
  config_vqgan = load_config(VQGAN_model, "/content/drive/MyDrive/GP/models_TT/%s.yaml" % VQGAN_model)
  model_vqgan  = load_vqgan(VQGAN_model, config_vqgan, ckpt_path="/content/drive/MyDrive/GP/models_TT/%s.ckpt" % VQGAN_model).cuda()
  # config_vqgan = load_config(VQGAN_model,"/content/models_TT/%s.yaml" % VQGAN_model)
  # model_vqgan  = load_vqgan(VQGAN_model,config_vqgan, ckpt_path="/content/models_TT/%s.ckpt" % VQGAN_model).cuda()
  # model_clip, _ = clip.load(model, jit=old_torch())
  # modsize = model_clip.visual.input_resolution
  # Set up the CLIP model we'll be using
  model_clip = clip.load(model, jit=False)[0].eval().requires_grad_(False).to(device)
  normalize = tf.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                  std=[0.26862954, 0.26130258, 0.27577711])
  modsize = model_clip.visual.input_resolution
  cutn=64
  cut_pow=1
  make_cutouts = MakeCutouts(modsize, cutn, cut_pow=cut_pow)
  model_lang = SentenceTransformer('clip-ViT-B-32-multilingual-v1').cuda()      # if we use ViT-B/32
  # model_lang = SentenceTransformer('clip-ViT-L-14').cuda()    # if we use ViT-L/14 or ViT-L/14@336px

  samples = int(samples * 0.25)
  if aug_transform == 'elastic':
    trform_f = transforms.transforms_elastic
    samples = int(samples * 0.95)
  elif aug_transform == 'custom':
    trform_f = transforms.transforms_custom
    samples = int(samples * 0.95)
  elif aug_transform == 'fast':
    trform_f = transforms.transforms_fast
    samples = int(samples * 0.95)
  else:
    trform_f = transforms.normalize()

  if len(style) > 0:
    txt_enc2 = enc_text(style, model_clip, model_lang, multilang)
    prompt = Prompt(txt_enc2, 1, float('-inf')).to(device)
  else:
    txt_enc2 = ""

  # create images.
  images = create_images(text, sideY, sideX, scale_res, model_clip, model_lang, multilang, learning_rate, steps, model_vqgan, save_freq, samples, modsize, trform_f, sign, txt_enc2, align , normalize , make_cutouts)
  
  # write on images.
  write_on_images(images, text, lang)
  
  # create video.
  audio_paths = create_audios(text, lang)
  video_clip_paths = create_videos(audio_paths)
  video = concatenate_videos(video_clip_paths)
  return images, video

In [ ]:
text = 'There are many landscapes such as, small waterfalls, forests, seas, birds in the sky, flowers in gardens, tall trees.'
style = "Classicism" # "Classicism", "Abstract Expressionism", "Art Deco", "Art Nouveau", "Baroque", "Fauvism", "Futurism", "Post-Impressionism", "Precisionism", "Land Art"
images, video = run_model(text, style)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

im = Image.open(images[0]) 
plt.imshow(im)